In [18]:
from pyspark.sql.functions import lower, regexp_replace, regexp_extract, col, trim, when, instr, lit, split, size
from pyspark.sql.functions import concat_ws
special_char = '[^a-z0-9A-Z_ ' \
               'àáãạảăắằẳẵặâấầẩẫậèéẹẻẽêềếểễệđìíĩỉịòóõọỏôốồổỗộơớờởỡợùúũụủưứừửữựỳỵỷỹýÀÁÃẠẢĂẮẰẲẴẶÂẤẦẨẪẬ' \
               'ÈÉẸẺẼÊỀẾỂỄỆĐÌÍĨỈỊÒÓÕỌỎÔỐỒỔỖỘƠỚỜỞỠỢÙÚŨỤỦƯỨỪỬỮỰỲỴỶỸÝ]+'

In [19]:
from pyspark.sql import SparkSession

# Create a DataFrame using SparkSession
spark = (SparkSession
    .builder
    .appName("data_cleaning")
    .enableHiveSupport()
    .getOrCreate())

In [20]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

path = '/home/jazzdung/data/all.ndjson'

# spark.conf.set("spark.sql.caseSensitive", True)
# df = spark.read.json(path)
# spark.conf.set("spark.sql.caseSensitive", False)

# to convert attrs to String
schema = StructType([
    StructField("attrs", StringType(), True),
    StructField("avg_rating", StringType(), True),
    StructField("num_review", StringType(), True),
    StructField("num_sold", StringType(), True),
    StructField("price", StringType(), True),
    StructField("product_desc", StringType(), True),
    StructField("product_name", StringType(), True),
    StructField("shipping", StringType(), True),
    StructField("shop_info", StringType(), True),
    StructField("url", StringType(), True)
])
df = spark.read.format("json").schema(schema)\
.load(path)


df.count()
# df.show(10)

109944

In [21]:

def k_to_number(c):  # 3,2k -> 3200  
    contain_comma = instr(c, ',') >= 1
    c = when(contain_comma, regexp_replace(c, 'k', '00')) \
        .otherwise(regexp_replace(c, 'k', '000'))
    c = regexp_replace(c, ',', '')
    return c

def clean_product_name(df):
    # Lowercase
    product_name = lower(col('product_name'))
    # Remove like tier
    product_name = regexp_replace(product_name, 'yêu thích\n|yêu thích\+\n', ' ')
    # Remove contents inside [], option indicate promotion, prices    
    product_name = regexp_replace(product_name, r'\[.*?\]', ' ')
    # Remove special character
    product_name = regexp_replace(product_name, special_char, ' ')
    # Remove redundant whitespaces  
    product_name = regexp_replace(product_name, ' +', ' ')

    # Trim
    product_name = trim(product_name)
    return df.withColumn('product_name', product_name)

def clean_price(df):
    value = col('price')   
    value = regexp_replace(value, r'.*₫' , '')
    value = regexp_replace(value, '\.', '')
    value = value.cast('int')

    return df.withColumn('price', value)

def clean_desc(df):
    product_desc = lower(col('product_desc'))
    product_desc = regexp_replace(product_desc, 'Vớ/ Tất', 'Vớ, Tất')
    product_desc = regexp_replace(product_desc, 'Vớ/Tất', 'Vớ, Tất')
    product_desc = regexp_replace(product_desc, 'Quần Dài/Quần Âu', 'Quần Dài, Quần Âu')
    product_desc = regexp_replace(product_desc, 'Quần Dài/ Quần Âu', 'Quần Dài, Quần Âu')
    product_desc = regexp_replace(product_desc, ' &amp;', ',')
    product_desc = regexp_replace(product_desc, '<svg.*?</svg>|<div>|div|class=|"|<label.*?>|<flex.*?>| href=/', '')
    product_desc = regexp_replace(product_desc, '</a>', '-')
    product_desc = regexp_replace(product_desc, '</label>', ': ')
    product_desc = regexp_replace(product_desc, '< ', '<')
    product_desc = regexp_replace(product_desc, "\/.*?\>","/>")
    product_desc = regexp_replace(product_desc, '<a ', '<')
    product_desc = regexp_replace(product_desc, '<p ', '<')
    product_desc = regexp_replace(product_desc, ' +', ' ')

    # Split
    product_desc = regexp_replace(product_desc, '</>', '/')
    product_desc = regexp_replace(product_desc, '<.*?>', '')

    return df.withColumn('product_desc', product_desc)

def extract_country(df):
    country = regexp_replace(col('product_desc'),  'mô tả sản phẩm(.*)' , '')
    country = regexp_extract(country, 'xuất xứ: (.+?)/', 1)
    country = regexp_replace(country, special_char, ' ')

    return df.withColumn('country', country)

def extract_brand(df):
    brand = regexp_replace(col('product_desc'),  'mô tả sản phẩm(.*)' , '')
    brand = regexp_extract(brand, 'thương hiệu: (.+?)-/', 1)
    brand = regexp_replace(brand, special_char, ' ')
    return df.withColumn('brand', brand)

def extract_stock(df):
    stock = regexp_replace(col('product_desc'),  'mô tả sản phẩm(.*)' , '')
    stock = regexp_extract(stock, 'kho hàng: (.+?)/', 1)
    stock = regexp_replace(stock, special_char, ' ')
    return df.withColumn('stock', stock)

def extract_origin(df):
    origin = regexp_replace(col('product_desc'),  'mô tả sản phẩm(.*)' , '')
    origin = regexp_extract(origin, 'gửi từ: (.+?)/', 1)
    return df.withColumn('origin', origin)

def extract_first_category(df):
    category = regexp_extract('product_desc', 'shopee-(.+?)-', 1)
    return df.withColumn('category', category)

def extract_second_category(df):
    category = regexp_extract('product_desc', 'shopee-(.+)-//', 1)
    cat_list = split(category, r"-")

    return df.withColumn('second_category', 
        when (
            size(cat_list) > 1,
            concat_ws(' / ',cat_list[0],cat_list[1])
        ).otherwise('no')
    )

def extract_third_category(df):
    category = regexp_extract('product_desc', 'shopee-(.+)-//', 1)
    cat_list = split(category, r"-")

    return df.withColumn('third_category', 
        when (
            size(cat_list) > 2,
            concat_ws(' / ',cat_list[0],cat_list[1], cat_list[2])
        ).otherwise('no')
    )
        

def extract_smaller_desc(df):
    description = regexp_extract('product_desc', 'mô tả sản phẩm(.*)', 1)
    description = regexp_replace(description, special_char, ' ')
    return df.withColumn('description', description)


def clean_attrs(df):
    attrs = lower(col('attrs'))
    attrs = regexp_replace(attrs, special_char, ' ')
    attrs = trim(attrs)
    return df.withColumn('attrs', attrs)

def extract_shop_name(df):
    remove_like_tier = regexp_replace(col('shop_info'), 'Yêu Thích\n|Yêu Thích\+\n', '')
    shop_name = regexp_extract(remove_like_tier, '(.+?)\n', 1)
    return df.withColumn('shop_name', shop_name)

def extract_shop_like_tier(df):
    shop_like_tier = regexp_extract(col('shop_info'), '^(Yêu Thích\+?)\n', 1)
    shop_like_tier = when(shop_like_tier == "Yêu Thích+", 2) \
        .when(shop_like_tier == "Yêu Thích", 1) \
        .otherwise(0)
    return df.withColumn('shop_like_tier', shop_like_tier)

def extract_shop_num_review(df):
    shop_num_review = regexp_extract(col('shop_info'), 'Đánh Giá\n(.+)\n', 1)
    shop_num_review = k_to_number(shop_num_review)
    shop_num_review = shop_num_review.cast('int')
    return df.withColumn('shop_num_review', shop_num_review)


def extract_shop_reply_percectage(df):
    shop_reply_percentage = regexp_extract(col('shop_info'), 'Tỉ Lệ Phản Hồi\n(.+)\n', 1)
    shop_reply_percentage = regexp_replace(shop_reply_percentage, '%', '')
    shop_reply_percentage = shop_reply_percentage.cast('float') / 100
    return df.withColumn('shop_reply_percentage', shop_reply_percentage)


def extract_shop_reply_time(df):
    shop_reply_time = regexp_extract(col('shop_info'), 'Thời Gian Phản Hồi\n(.+)\n', 1)
    return df.withColumn("shop_reply_time", shop_reply_time)


def extract_shop_creation_time(df):
    shop_creation_time = regexp_extract(col('shop_info'), 'Tham Gia\n(.+)\n', 1)
    num = regexp_extract(shop_creation_time, '\d+', 0).cast('int')
    contain_year = instr(shop_creation_time, 'năm') >= 1
    contain_month = instr(shop_creation_time, 'tháng') >= 1
    shop_creation_time = when(contain_year, num * 12) \
        .otherwise(when(contain_month, num) \
                   .otherwise(lit(0)))
    return df.withColumn("shop_creation_time", shop_creation_time)


def extract_shop_num_follower(df):
    shop_num_follower = regexp_extract(col('shop_info'), 'Người Theo Dõi\n(.+)', 1)
    shop_num_follower = k_to_number(shop_num_follower)
    shop_num_follower = shop_num_follower.cast('int')
    return df.withColumn("shop_num_follower", shop_num_follower)


def clean_shipping(df):
    shipping = lower(col('shipping'))
    shipping = regexp_replace(shipping, special_char, '')
    # shipping = regexp_extract(shipping, r'\d+', 0)
    return df.withColumn('shipping', shipping)

def clean_numeric_field(df, col_name):
    cleaned_field = k_to_number(col(col_name))
    cleaned_field = cleaned_field.cast('int')
    return df.withColumn(col_name, cleaned_field)

In [22]:
# cleaned product name
df_cleaned = clean_product_name(df)

# Cleaned price
df_cleaned = clean_price(df_cleaned)

# Clean product description
df_cleaned = clean_desc(df_cleaned)

# Get country, brand, stocks, origin
df_cleaned = extract_country(df_cleaned)
df_cleaned = extract_brand(df_cleaned)
df_cleaned = extract_stock(df_cleaned)
df_cleaned = extract_origin(df_cleaned)

# Get category
df_cleaned = extract_first_category(df_cleaned)
df_cleaned = extract_second_category(df_cleaned)
df_cleaned = extract_third_category(df_cleaned)

# Desc
df_cleaned = extract_smaller_desc(df_cleaned)

# Attrs
df_cleaned = clean_attrs(df_cleaned)

# Shop info
if "shop_info" in df_cleaned.columns:
    df_cleaned = extract_shop_name(df_cleaned)
    df_cleaned = extract_shop_like_tier(df_cleaned)
    df_cleaned = extract_shop_num_review(df_cleaned)
    df_cleaned = extract_shop_reply_percectage(df_cleaned)
    df_cleaned = extract_shop_reply_time(df_cleaned)
    df_cleaned = extract_shop_creation_time(df_cleaned)
    df_cleaned = extract_shop_num_follower(df_cleaned)

    # Shipping
    df_cleaned = clean_shipping(df_cleaned)

# Number of product review
df_cleaned = clean_numeric_field(df_cleaned, "num_sold")

# Number of product sold
df_cleaned = clean_numeric_field(df_cleaned, "num_review")

# Drop redundant columns
df_cleaned = df_cleaned.drop('product_desc')
df_cleaned = df_cleaned.drop('shop_info')


# df_cleaned.show(1000, truncate=False)

In [23]:
# df_cleaned = df_cleaned.withColumn("price",df_cleaned["price"].cast('int'))
# df_cleaned = df_cleaned.withColumn("num_review",df_cleaned["num_review"].cast('int'))
# df_cleaned = df_cleaned.withColumn("num_sold",df_cleaned["num_sold"].cast('int'))
# df_cleaned = df_cleaned.withColumn("stock",df_cleaned["stock"].cast('int'))
df_cleaned = df_cleaned.withColumn("avg_rating",df_cleaned["avg_rating"].cast('double'))


# if "shop_info" in df_cleaned.columns:
#     df_cleaned = df_cleaned.withColumn("shop_like_tier",df_cleaned["shop_like_tier"].cast('int'))
#     df_cleaned = df_cleaned.withColumn("shop_num_review",df_cleaned["shop_num_review"].cast('int'))
#     df_cleaned = df_cleaned.withColumn("shop_num_follower",df_cleaned["shop_num_follower"].cast('int'))
#     df_cleaned = df_cleaned.withColumn("shop_creation_time",df_cleaned["shop_creation_time"].cast('int'))
#     df_cleaned = df_cleaned.withColumn("shop_reply_percentage",df_cleaned["shop_reply_percentage"].cast('double'))
    
df_cleaned.printSchema()

root
 |-- attrs: string (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- num_review: integer (nullable = true)
 |-- num_sold: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- shipping: string (nullable = true)
 |-- url: string (nullable = true)
 |-- country: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- stock: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- category: string (nullable = true)
 |-- second_category: string (nullable = false)
 |-- third_category: string (nullable = false)
 |-- description: string (nullable = true)
 |-- shop_name: string (nullable = true)
 |-- shop_like_tier: integer (nullable = false)
 |-- shop_num_review: integer (nullable = true)
 |-- shop_reply_percentage: double (nullable = true)
 |-- shop_reply_time: string (nullable = true)
 |-- shop_creation_time: integer (nullable = true)
 |-- shop_num_follower: integer (nullable = true)



In [24]:
from pyspark.sql import Window
from pyspark.sql.functions import col, row_number

# Match product name, get highest price
w = Window.partitionBy('product_name').orderBy(col("price").desc())

df_cleaned = df_cleaned.withColumn("row",row_number().over(w)).filter(col("row") == 1).drop("row")

# df_cleaned.show(10)

+--------------------+----------+----------+--------+------+--------------------+--------------------+--------------------+--------+---------+------+--------------------+-----------------+--------------------+--------------------+--------------------+-----------------+--------------+---------------+---------------------+---------------+------------------+-----------------+
|               attrs|avg_rating|num_review|num_sold| price|        product_name|            shipping|                 url| country|    brand| stock|              origin|         category|     second_category|      third_category|         description|        shop_name|shop_like_tier|shop_num_review|shop_reply_percentage|shop_reply_time|shop_creation_time|shop_num_follower|
+--------------------+----------+----------+--------+------+--------------------+--------------------+--------------------+--------+---------+------+--------------------+-----------------+--------------------+--------------------+------------------

In [25]:
(df_cleaned  
    .coalesce(1)
    .write.option("header", True)
    .format("csv")
    .mode('overwrite')
    .csv("/home/jazzdung/data/all_with_shop_name.csv"))